# Total Grid Forecasting
Here, load values are forecasted for 1st week of July 2008 (all 20 zones).
> SARIMAX is used to forecast temperature values.

## Pre-processing

### Defining Dependent Variable
Both actual load values and its log transformation are defined here.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

# Loading the data
load_long = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 6 (TU Dresden)\Thesis Work\Exploratory Code\load_history_long.csv").sort_values(by = "timestamp")

# Converting to Wide Format
load_wide = load_long.pivot_table(
    index='timestamp',      # The column to use as the index
    columns='zone_id',  # The column whose unique values will become the new column names
    values='load'    # The column to use for the values in the new DataFrame.
).sort_values(by="timestamp")

# Converting string to datetime
from datetime import datetime
load_wide.index = pd.to_datetime(load_wide.index)
load_wide.index[0]

# Segregating temporal information
load_wide['year'] = load_wide.index.year
load_wide['month'] = load_wide.index.month
load_wide['day'] = load_wide.index.day
load_wide['hour'] = load_wide.index.hour

load_wide.head()

zone_id,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,year,month,day,hour
timestamp,,,,,,,,,,,,,,,,,,,,,
2004-01-01 00:30:00,16853.0,126259.0,136233.0,484.0,6829.0,133088.0,136233.0,3124.0,75243.0,23339.0,...,65970.0,28752.0,30645.0,200946.0,82298.0,79830.0,2004,1,1,0
2004-01-01 01:30:00,16450.0,123313.0,133055.0,457.0,6596.0,129909.0,133055.0,2956.0,67368.0,22100.0,...,64600.0,27851.0,30461.0,195835.0,79827.0,77429.0,2004,1,1,1
2004-01-01 02:30:00,16517.0,119192.0,128608.0,450.0,6525.0,125717.0,128608.0,2953.0,64050.0,21376.0,...,63843.0,27631.0,30197.0,194093.0,77728.0,75558.0,2004,1,1,2
2004-01-01 03:30:00,16873.0,117507.0,126791.0,448.0,6654.0,124162.0,126791.0,2914.0,63861.0,21335.0,...,64023.0,27986.0,30264.0,194708.0,76433.0,75709.0,2004,1,1,3
2004-01-01 04:30:00,17064.0,118343.0,127692.0,444.0,6977.0,125320.0,127692.0,3221.0,75852.0,21564.0,...,65679.0,29160.0,30907.0,202458.0,78172.0,77475.0,2004,1,1,4


In [2]:
# Log transformation on load values (no-scaling)
load_wide_log = load_wide
load_wide_log[list(range(1,21,1))] = load_wide_log[list(range(1,21,1))].apply(np.log)
load_wide_log[0:2]

zone_id,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,year,month,day,hour
timestamp,,,,,,,,,,,,,,,,,,,,,
2004-01-01 00:30:00,9.732284,11.746091,11.822122,6.182085,8.828934,11.798766,11.822122,8.046870,11.228478,10.057881,...,11.096955,10.266463,10.330225,12.210791,11.318102,11.287655,2004,1,1,0
2004-01-01 01:30:00,9.708081,11.722481,11.798518,6.124683,8.794219,11.774589,11.798518,7.991592,11.117925,10.003333,...,11.075970,10.234624,10.324202,12.185028,11.287617,11.257117,2004,1,1,1


### Defining and Forecasting Temperature
SARIMAX is performed to forecast temperature values, to be used as exogenous variable for load forecasting.
> Training period for temperature values using SARIMAX is roughly 2 weeks, as done for benchmarking models in temperature forecasting notebook.

In [3]:
# Reading data
temp = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 6 (TU Dresden)\Thesis Work\Exploratory Code\weighted_temperature.csv")
temp = temp[["timestamp", "temp_weighted"]]
temp[0:2]

,timestamp,temp_weighted
0,2004-01-01 00:30:00,42.338937
1,2004-01-01 01:30:00,41.239284


In [12]:
# Defining training data for SARIMAX
temp_train_set = temp.loc[(temp.timestamp >= "2008-06-16 00:30:00") & (temp.timestamp <= "2008-06-30 23:30:00"),:]

# Defining one harmonic - on training set
from statsmodels.tsa.deterministic import Fourier
periodicity = Fourier(period=24, order=1) # daily cycle i.e. 24 hours, 1 harmonic

from statsmodels.tsa.deterministic import DeterministicProcess
dp = DeterministicProcess(
                        index=temp_train_set.timestamp,
                        period=None,         # It's not defined so that frequency can be read from the index
                        constant=False,      # Let regression model define this intercept later on
                        order=0,             # linear trend is excluded 
                        seasonal=False,      # no seasonal dummies
                        additional_terms=[periodicity], # 1 individual wave will be generated
                        drop=True            # if perfect collinearity exists, the terms can be dropped
                        )
            
wave = dp.in_sample()
temp_train_set = pd.merge(temp_train_set, wave, on="timestamp", how="inner")
temp_train_set[0:2]

,timestamp,temp_weighted,"sin(1,24)","cos(1,24)"
0,2008-06-16 00:30:00,67.426590,0.000000,1.000000
1,2008-06-16 01:30:00,66.241917,0.258819,0.965926


In [18]:
# Defining temperature values
temp_forecast_set = pd.DataFrame(
    np.array(pd.date_range(start="2008-07-01 00:30:00", end="2008-07-07 23:30:00", freq='h')), 
    columns=['timestamp']
)

# Defining one harmonic - on forecast set
dp = DeterministicProcess(
                        index=temp_forecast_set.timestamp,
                        period=None,         # It's not defined so that frequency can be read from the index
                        constant=False,      # Let regression model define this intercept later on
                        order=0,             # linear trend is excluded 
                        seasonal=False,      # no seasonal dummies
                        additional_terms=[periodicity], # 1 individual wave will be generated
                        drop=True            # if perfect collinearity exists, the terms can be dropped
                        )
            
wave = dp.in_sample()
temp_forecast_set = pd.merge(temp_forecast_set, wave, on="timestamp", how="inner")
temp_forecast_set[0:2]

,timestamp,"sin(1,24)","cos(1,24)"
0,2008-07-01 00:30:00,0.000000,1.000000
1,2008-07-01 01:30:00,0.258819,0.965926


In [20]:
# Training multiple SARIMAX models to find correct orders
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm

smodel = pm.auto_arima(y = temp_train_set.temp_weighted, 
                       X = temp_train_set[['sin(1,24)', 'cos(1,24)']],
                       start_p=0, start_q=0,test='adf',
                       max_p=3, max_q=3, m=24,
                       start_P=0, start_Q=0, max_P=3, max_Q=3, seasonal=True,
                       d=0, D=0, trace=True,
                       trend='c',
                       error_action='ignore',  
                       suppress_warnings=True, 
                       stepwise=True
                     )

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[24] intercept   : AIC=2071.496, Time=0.04 sec
 ARIMA(1,0,0)(1,0,0)[24] intercept   : AIC=935.917, Time=1.54 sec
 ARIMA(0,0,1)(0,0,1)[24] intercept   : AIC=1517.512, Time=0.77 sec
 ARIMA(0,0,0)(0,0,0)[24]             : AIC=2071.496, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[24] intercept   : AIC=1087.413, Time=0.10 sec
 ARIMA(1,0,0)(2,0,0)[24] intercept   : AIC=920.444, Time=3.67 sec
 ARIMA(1,0,0)(3,0,0)[24] intercept   : AIC=inf, Time=10.10 sec
 ARIMA(1,0,0)(2,0,1)[24] intercept   : AIC=inf, Time=6.23 sec
 ARIMA(1,0,0)(1,0,1)[24] intercept   : AIC=893.596, Time=2.31 sec
 ARIMA(1,0,0)(0,0,1)[24] intercept   : AIC=999.900, Time=1.20 sec
 ARIMA(1,0,0)(1,0,2)[24] intercept   : AIC=inf, Time=6.54 sec
 ARIMA(1,0,0)(0,0,2)[24] intercept   : AIC=955.071, Time=3.43 sec
 ARIMA(1,0,0)(2,0,2)[24] intercept   : AIC=inf, Time=7.26 sec
 ARIMA(0,0,0)(1,0,1)[24] intercept   : AIC=1827.719, Time=1.96 sec
 ARIMA(2,0,0)(1,0,1)[24] intercept   : AIC=

In [23]:
# Fitting the model
import statsmodels.api as sm
SARIMA_model = sm.tsa.statespace.SARIMAX(
        endog = temp_train_set.temp_weighted, 
        exog = temp_train_set[['sin(1,24)', 'cos(1,24)']],
        order=(2,0,0),  # non-seasonal: with no differencing
        seasonal_order=(1,0,1,24), # seasonal: with no seasonal differencing
        trend="c"
)

SARIMA_model_fit = SARIMA_model.fit(disp=False, maxiter=600)

# Forecasting
temp_forecast = np.array(
    SARIMA_model_fit.get_forecast(
        steps=len(temp_forecast_set), 
        exog = temp_forecast_set[['sin(1,24)', 'cos(1,24)']]
    ).predicted_mean
)

temp_forecast[0:3]

C:\Users\singh\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:997: UserWarning: Non-stationary starting seasonal autoregressive Using zeros as starting parameters.
  warn('Non-stationary starting seasonal autoregressive'


array([68.48787973, 68.39339955, 68.74240037])

### Defining Feature Matrix
Here exogenous variables for load forecasting are defined. Temperature information is already present.

##### Training

In [27]:
# Adding temperature variable - training
feature_matrix = temp_train_set[["timestamp","temp_weighted"]]
feature_matrix.set_index("timestamp", inplace=True)
feature_matrix.index = pd.to_datetime(feature_matrix.index)
feature_matrix[0:3]

,temp_weighted
timestamp,
2008-06-16 00:30:00,67.426590
2008-06-16 01:30:00,66.241917
2008-06-16 02:30:00,65.158480


Adding holiday information won't help, since training isn't big enough to 'inform' variable values on a "holiday".

In [28]:
# Adding temperature knots for PLR

T_H = 55  # Heating Threshold
T_C = 65  # Cooling Threshold

# Temporarily changing alias of df
mul_df = feature_matrix.copy()

# Construct the Heating Demand Knot: HDK = max(0, T_H - Temp)
## This captures load increase when temp is below T_H.
mul_df["HDK"] = np.where(
    mul_df["temp_weighted"] < T_H,  
    T_H - mul_df["temp_weighted"],  # Value if True: The positive difference
    0                               # Value if False: Zero
)

# Construct the Cooling Demand Knot; CDK = max(0, Temp - T_C)
## This captures load increase when temp is above T_C.
mul_df["CDK"] = np.where(
    mul_df["temp_weighted"] > T_C,  
    mul_df["temp_weighted"] - T_C,  # Value if True: The positive difference
    0                               # Value if False: Zero
)

# Reverting back to original alias
feature_matrix = mul_df.copy()
feature_matrix[0:3]

,temp_weighted,HDK,CDK
timestamp,,,
2008-06-16 00:30:00,67.426590,0.0,2.426590
2008-06-16 01:30:00,66.241917,0.0,1.241917
2008-06-16 02:30:00,65.158480,0.0,0.158480


In [29]:
# Adding harmonoics to feature matrix
periodicity = Fourier(period=24, order=2) # daily cycle i.e. 24 hours, 2 harmonics

# Defining the sine wave
dp = DeterministicProcess(
        index=feature_matrix.index,
        period=None,         # It's not defined so that frequency can be read from the index
        constant=False,      # defined later
        order=1,             # linear trend added
        seasonal=False,      # no seasonal dummies
        additional_terms=[periodicity], # 2 seperate waves will be generated
        drop=True            # if perfect collinearity exists, the terms can be dropped
)

waves = dp.in_sample()

# Merging to feature matrix
feature_matrix = feature_matrix.merge(waves, left_index=True, right_index=True, how='left')
feature_matrix[0:2]

,temp_weighted,HDK,CDK,trend,"sin(1,24)","cos(1,24)","sin(2,24)","cos(2,24)"
timestamp,,,,,,,,
2008-06-16 00:30:00,67.426590,0.0,2.426590,1.0,0.000000,1.000000,0.0,1.000000
2008-06-16 01:30:00,66.241917,0.0,1.241917,2.0,0.258819,0.965926,0.5,0.866025


In [46]:
# Specifying exogenous and endogenous variables
X_train = feature_matrix[['CDK', 'HDK', 'trend', 'sin(1,24)', 'cos(1,24)', 'sin(2,24)', 'cos(2,24)']]
y_train = load_wide_log.loc[temp_train_set.timestamp, list(range(1,21,1))]

##### Prediction Features

In [36]:
# Adding temperature variable - prediction set
feature_matrix_pred = pd.DataFrame()
feature_matrix_pred["temp_predicted"] = temp_forecast
feature_matrix_pred.index = pd.to_datetime(temp_forecast_set["timestamp"])
feature_matrix_pred[0:3]

,temp_predicted
timestamp,
2008-07-01 00:30:00,68.48788
2008-07-01 01:30:00,68.39340
2008-07-01 02:30:00,68.74240


In [37]:
# Adding temperature knots for PLR

T_H = 55  # Heating Threshold
T_C = 65  # Cooling Threshold

# Temporarily changing alias of df
mul_df = feature_matrix_pred.copy()

# Construct the Heating Demand Knot: HDK = max(0, T_H - Temp)
## This captures load increase when temp is below T_H.
mul_df["HDK"] = np.where(
    mul_df["temp_predicted"] < T_H,  
    T_H - mul_df["temp_predicted"],  # Value if True: The positive difference
    0                               # Value if False: Zero
)

# Construct the Cooling Demand Knot; CDK = max(0, Temp - T_C)
## This captures load increase when temp is above T_C.
mul_df["CDK"] = np.where(
    mul_df["temp_predicted"] > T_C,  
    mul_df["temp_predicted"] - T_C,  # Value if True: The positive difference
    0                               # Value if False: Zero
)

# Reverting back to original alias
feature_matrix_pred = mul_df.copy()
feature_matrix_pred[0:3]

,temp_predicted,HDK,CDK
timestamp,,,
2008-07-01 00:30:00,68.48788,0.0,3.48788
2008-07-01 01:30:00,68.39340,0.0,3.39340
2008-07-01 02:30:00,68.74240,0.0,3.74240


In [38]:
# Defining the sine wave
dp = DeterministicProcess(
        index=feature_matrix_pred.index,
        period=None,         # It's not defined so that frequency can be read from the index
        constant=False,      # defined later
        order=1,             # linear trend added
        seasonal=False,      # no seasonal dummies
        additional_terms=[periodicity], # 2 seperate waves will be generated
        drop=True            # if perfect collinearity exists, the terms can be dropped
)

waves = dp.in_sample()

# Merging to feature matrix pred
feature_matrix_pred = feature_matrix_pred.merge(waves, left_index=True, right_index=True, how='left')
feature_matrix_pred[0:2]

,temp_predicted,HDK,CDK,trend,"sin(1,24)","cos(1,24)","sin(2,24)","cos(2,24)"
timestamp,,,,,,,,
2008-07-01 00:30:00,68.48788,0.0,3.48788,1.0,0.000000,1.000000,0.0,1.000000
2008-07-01 01:30:00,68.39340,0.0,3.39340,2.0,0.258819,0.965926,0.5,0.866025


In [39]:
# Specifying exogenous variables
X_forecast_set = feature_matrix_pred[['CDK', 'HDK', 'trend', 'sin(1,24)', 'cos(1,24)', 'sin(2,24)', 'cos(2,24)']]

### Forecasting
Performed for first week of July 2008: from 1 July 2008 (h1) to 7 July 2008 (h24).

#### Time Series Regression (With Harmonics + Piecewise Temperature segments)

In [47]:
# Fitting Linear Regression Model (all zones) on training set 
from sklearn.linear_model import LinearRegression

# Creating dictionary to store the fitted models
fitted_models = {}

print("Starting model training for complete grid...")

# Looping through each target column
for zone in list(y_train.columns):
    
    # Extract the current target vector (y)
    y = y_train[zone]
    
    # 1. Instantiate the model
    # A new model object is created for each iteration
    model = LinearRegression()
    
    # 2. Fit the model
    # Train the model using the common features (X) and the current target (y)
    model.fit(X_train, y)
    
    # 3. Store the fitted model in the dictionary
    fitted_models[zone] = model
    
    print(f"  -> Finished fitting model for: {zone}")

print("Training complete for the entire grid!")

Starting model training for complete grid...
  -> Finished fitting model for: 1
  -> Finished fitting model for: 2
  -> Finished fitting model for: 3
  -> Finished fitting model for: 4
  -> Finished fitting model for: 5
  -> Finished fitting model for: 6
  -> Finished fitting model for: 7
  -> Finished fitting model for: 8
  -> Finished fitting model for: 9
  -> Finished fitting model for: 10
  -> Finished fitting model for: 11
  -> Finished fitting model for: 12
  -> Finished fitting model for: 13
  -> Finished fitting model for: 14
  -> Finished fitting model for: 15
  -> Finished fitting model for: 16
  -> Finished fitting model for: 17
  -> Finished fitting model for: 18
  -> Finished fitting model for: 19
  -> Finished fitting model for: 20
Training complete for the entire grid!


In [69]:
# Predicting load value per zone - from 20 fitted models each

# Initializing an empty DataFrame with the correct index
predictions_set = pd.DataFrame(index=X_forecast_set.index)

# Initialising message
print("Initialising prediction generation...")

# Loop through the dictionary items
for zone, model in fitted_models.items():
    
    # 1. Generate Predictions
    # This returns a NumPy array of predicted values
    predictions_array = model.predict(X_forecast_set)
    
    # 2. Assign the predictions array as a new column
    # The new column is named 'Predicted_Target_X'
    column_name = f'Zone_{zone}_pred'
    
    # Pandas should match the array to the DataFrame's existing index
    predictions_set[column_name] = predictions_array
    
    print(f" -> {column_name} added as a column")

print("Prediction generation for the entire grid complete!")

Initialising prediction generation...
 -> Zone_1_pred added as a column
 -> Zone_2_pred added as a column
 -> Zone_3_pred added as a column
 -> Zone_4_pred added as a column
 -> Zone_5_pred added as a column
 -> Zone_6_pred added as a column
 -> Zone_7_pred added as a column
 -> Zone_8_pred added as a column
 -> Zone_9_pred added as a column
 -> Zone_10_pred added as a column
 -> Zone_11_pred added as a column
 -> Zone_12_pred added as a column
 -> Zone_13_pred added as a column
 -> Zone_14_pred added as a column
 -> Zone_15_pred added as a column
 -> Zone_16_pred added as a column
 -> Zone_17_pred added as a column
 -> Zone_18_pred added as a column
 -> Zone_19_pred added as a column
 -> Zone_20_pred added as a column
Prediction generation for the entire grid complete!


In [70]:
# Undoing log transformation for original predictions
predictions_set_unlogged = np.exp(predictions_set)
predictions_set_unlogged[:2]

,Zone_1_pred,Zone_2_pred,Zone_3_pred,Zone_4_pred,Zone_5_pred,Zone_6_pred,Zone_7_pred,Zone_8_pred,Zone_9_pred,Zone_10_pred,Zone_11_pred,Zone_12_pred,Zone_13_pred,Zone_14_pred,Zone_15_pred,Zone_16_pred,Zone_17_pred,Zone_18_pred,Zone_19_pred,Zone_20_pred
timestamp,,,,,,,,,,,,,,,,,,,,
2008-07-01 00:30:00,15263.155937,140930.919112,152064.662365,327.253745,4756.515831,145682.885209,152064.662365,2438.210554,87182.690057,68475.513183,89782.498732,108154.664944,14431.770225,17883.410087,49131.091993,21226.628544,30883.924829,161193.002434,64218.086532,72807.809942
2008-07-01 01:30:00,13821.078871,132614.477544,143091.214158,298.540855,4267.725062,136845.149184,143091.214158,2251.376437,84295.887239,64351.106516,82937.171580,98322.776476,13179.170682,15831.366566,45222.267874,19226.718296,28835.856705,145092.812434,57662.825876,67749.469383


#### Naive Method: Last Cycle Prediction
As a baseline, last cycle load values are used as predicted values for the forecasting horizon.
> Note: In the training set, reading for load values are absent after the 6th hour of 30 June 2008!

In [60]:
# Repeating last cycle (unlogged)
last_cycle_naive_unlogged = np.exp(
    y_train.loc[(y_train.index < "2008-06-30 00:30:00") & (y_train.index >= "2008-06-23 00:30:00"), :]
)

# Reset index (correcting timestamp)
last_cycle_naive_unlogged.index = X_forecast_set.index
last_cycle_naive_unlogged[0:2]

zone_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
timestamp,,,,,,,,,,,,,,,,,,,,
2008-07-01 00:30:00,13024.0,132237.0,142685.0,307.0,4670.0,136907.0,142685.0,2345.0,59535.0,63461.0,91706.0,123113.0,13459.0,17453.0,45029.0,20862.0,30899.0,160212.0,58627.0,64829.0
2008-07-01 01:30:00,11842.0,125368.0,135272.0,295.0,4219.0,129587.0,135272.0,2200.0,59346.0,60692.0,84105.0,110222.0,12515.0,15732.0,41673.0,19166.0,29302.0,145365.0,53884.0,61705.0


### Performance Comparison
- Time Series Regression forecasts are compared with actual load values.
- In addition, Naive forecasts and benchmark forecast values are also compared with actual load values.

#### Reading Actual Values

In [63]:
# Reading actual load values
actual_load = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 6 (TU Dresden)\Thesis Work\Dataset\GEFCom2012\GEFCOM2012_Data\Load\Load_solution.csv")
actual_load.drop(["id","weight"], axis=1, inplace=True)

# Converting the data into long-format
actual_load_long = actual_load.melt(
                id_vars=["zone_id","year","month","day"],
                value_vars=[f"h{i}" for i in range(1, 25)],
                var_name="hour",
                value_name="load"
                        )

# Replacing hour values with interval mid-point
## Create a mapping from 'h1' to 'h24' → '00:30' to '23:30'
hour_map = {f"h{i}": f"{str(i-1).zfill(2)}:30" for i in range(1, 25)}

# Replace the values using .map()
actual_load_long["hour"] = actual_load_long["hour"].map(hour_map)

# Creating timestamps using existing information
actual_load_long["timestamp"] = pd.to_datetime(
    actual_load_long["year"].astype(str) + "-" +
    actual_load_long["month"].astype(str).str.zfill(2) + "-" +
    actual_load_long["day"].astype(str).str.zfill(2) + " " +
    actual_load_long["hour"]
)

actual_load_long.head()

,zone_id,year,month,day,hour,load,timestamp
0,1,2005,3,6,00:30,19964,2005-03-06 00:30:00
1,2,2005,3,6,00:30,162096,2005-03-06 00:30:00
2,3,2005,3,6,00:30,174901,2005-03-06 00:30:00
3,4,2005,3,6,00:30,528,2005-03-06 00:30:00
4,5,2005,3,6,00:30,9061,2005-03-06 00:30:00


In [65]:
# Trimming actual load values to only contain forecasting horizon
actual_load_long = actual_load_long.loc[(actual_load_long.year == 2008) & (actual_load_long.month == 7),:]

# Pivoting to record zone by column
actual_load_long = pd.pivot_table(actual_load_long, index="timestamp", columns="zone_id", values="load")
actual_load_long[0:2]

zone_id,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
timestamp,,,,,,,,,,,,,,,,,,,,,
2008-07-01 00:30:00,12037.0,136974.0,147795.0,311.0,4791.0,141765.0,147795.0,2292.0,75621.0,78323.0,...,119149.0,13247.0,19511.0,51361.0,23727.0,30319.0,170323.0,60293.0,72710.0,1407356.0
2008-07-01 01:30:00,10911.0,131584.0,141980.0,301.0,4215.0,135799.0,141980.0,2138.0,76314.0,72525.0,...,105920.0,12282.0,16907.0,47422.0,21106.0,27347.0,151755.0,54248.0,67957.0,1314493.0


#### Reading Load Benchmark Values

In [77]:
# Reading benchmark load

load_bench = pd.read_csv(r"C:\Users\singh\Desktop\TUD (All Semesters)\Courses - Semester 6 (TU Dresden)\Thesis Work\Dataset\GEFCom2012\GEFCOM2012_Data\Load\Load_benchmark.csv")

# Converting the data into long-format
load_bench = load_bench.melt(
                id_vars=["zone_id","year","month","day"],
                value_vars=[f"h{i}" for i in range(1, 25)],
                var_name="hour",
                value_name="load"
                        )


# Replacing hour values with interval mid-point

# Replace the values using .map()
load_bench["hour"] = load_bench["hour"].map(hour_map)

# Creating timestamps using existing information
load_bench["timestamp"] = pd.to_datetime(
    load_bench["year"].astype(str) + "-" +
    load_bench["month"].astype(str).str.zfill(2) + "-" +
    load_bench["day"].astype(str).str.zfill(2) + " " +
    load_bench["hour"]
)

# Trimming benchmark load values to only contain forecasting horizon
load_bench = load_bench.loc[(load_bench.year == 2008) & (load_bench.month == 7),:]

# Pivoting to record zone by column
load_bench = pd.pivot_table(load_bench, index="timestamp", columns="zone_id", values="load")
load_bench[0:2]

zone_id,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
timestamp,,,,,,,,,,,,,,,,,,,,,
2008-07-01 00:30:00,16165.0,151110.0,163047.0,345.0,5484.0,156594.0,163047.0,2712.0,81380.0,41052.0,...,112572.0,14818.0,18287.0,54974.0,24078.0,31226.0,177469.0,66554.0,76152.0,1450775.0
2008-07-01 01:30:00,14756.0,142943.0,154235.0,322.0,4846.0,147789.0,154235.0,2534.0,81471.0,38983.0,...,108726.0,13676.0,16331.0,50637.0,21928.0,29062.0,161349.0,60072.0,71726.0,1367162.0


#### Comparing Actual Values with Time Series Regression

In [81]:
# Initiating empty lists
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
r2_scores_tsr = []
mape_scores_tsr = []
rmse_scores_tsr = []

for i in range(1,21,1):
    # calculating score metrics for each zone 'i'
    r2 = r2_score(np.array(actual_load_long[i]),np.array(predictions_set_unlogged[f'Zone_{i}_pred']))
    mape = mean_absolute_percentage_error(np.array(actual_load_long[i]),np.array(predictions_set_unlogged[f'Zone_{i}_pred']))
    rmse = np.sqrt(mean_squared_error(np.array(actual_load_long[i]),np.array(predictions_set_unlogged[f'Zone_{i}_pred'])))

    # adding scores to score list
    r2_scores_tsr.append(r2)
    mape_scores_tsr.append(mape)
    rmse_scores_tsr.append(rmse)

print(f"Average R2 score for all zones, Time Series Regression: {np.mean(r2_scores_tsr)}")
print(f"Average MAPE score for all zones, Time Series Regression: {np.mean(mape_scores_tsr)}")
print(f"Average RMSE score for all zones, Time Series Regression: {np.mean(rmse_scores_tsr)}")

Average R2 score for all zones, Time Series Regression: 0.7059564294259296
Average MAPE score for all zones, Time Series Regression: 0.11708923982124941
Average RMSE score for all zones, Time Series Regression: 11155.946825510628


#### Comparing Actual Values with Naive Method

In [83]:
# Initiating empty lists
from sklearn.metrics import mean_absolute_percentage_error, r2_score
r2_scores_naive = []
mape_scores_naive = []
rmse_scores_naive = []

for i in range(1,21,1):
    # calculating score metrics for each zone 'i'
    r2 = r2_score(np.array(actual_load_long[i]),np.array(last_cycle_naive_unlogged[i]))
    mape = mean_absolute_percentage_error(np.array(actual_load_long[i]),np.array(last_cycle_naive_unlogged[i]))
    rmse = np.sqrt(mean_squared_error(np.array(actual_load_long[i]),np.array(last_cycle_naive_unlogged[i])))

    # adding scores to score list
    r2_scores_naive.append(r2)
    mape_scores_naive.append(mape)
    rmse_scores_naive.append(rmse)

print(f"Average R2 score for all zones, Naive Method: {np.mean(r2_scores_naive)}")
print(f"Average MAPE score for all zones, Naive Method: {np.mean(mape_scores_naive)}")
print(f"Average RMSE score for all zones, Naive Method: {np.mean(rmse_scores_naive)}")

Average R2 score for all zones, Naive Method: 0.4302329887190986
Average MAPE score for all zones, Naive Method: 0.1501921565865224
Average RMSE score for all zones, Naive Method: 17127.924335473843


#### Comparing Actual Values with Load Benchmark Values

In [84]:
# Initiating empty lists
from sklearn.metrics import mean_absolute_percentage_error, r2_score
r2_scores_bench = []
mape_scores_bench = []
rmse_scores_bench = []

for i in range(1,21,1):
    # calculating score metrics for each zone 'i'
    r2 = r2_score(np.array(actual_load_long[i]),np.array(load_bench[i]))
    mape = mean_absolute_percentage_error(np.array(actual_load_long[i]),np.array(load_bench[i]))
    rmse = np.sqrt(mean_squared_error(np.array(actual_load_long[i]),np.array(load_bench[i])))

    # adding scores to score list
    r2_scores_bench.append(r2)
    mape_scores_bench.append(mape)
    rmse_scores_bench.append(rmse)

print(f"Average R2 score for all zones, Benchmarked Load: {np.mean(r2_scores_bench)}")
print(f"Average MAPE score for all zones, Benchmarked Load: {np.mean(mape_scores_bench)}")
print(f"Average RMSE score for all zones, Benchmarked Load: {np.mean(rmse_scores_bench)}")

Average R2 score for all zones, Benchmarked Load: 0.32498790260757476
Average MAPE score for all zones, Benchmarked Load: 0.1753193209524572
Average RMSE score for all zones, Benchmarked Load: 15513.587923785488


### Conclusion

NOTE: Gefcom 2012 compares user model performanace wrt competition benchmark, not user defined naive method.

For Forecasting:
>- From Naive to Competition Benchmark, error reduction happened by -16% (MAPE)
>- <b>From Naive to Competition Benchmark, error reduction happened by 9.5% (RMSE)</b>

>- From Naive to Time Series Regression, error reduction happened by 22% (MAPE)
>- <b>From Naive to Time Series Regression, error reduction happened by 34.8% (RMSE)</b>

For Forecasting combined with Missing Values:
>- <b>From Naive to Competition Benchmark, error reduction happened on average by 34.4% (RMSE)</b>
>- <b>From Naive to Time Series Regression, error reduction happened on average by 43% (RMSE)</b>

#### Comparing Time Series Reg model to benchmarked values:
- Error i.e. RMSE reduced from 15513.587923785488 to 11155.946825510628: <b>28.1%</b>
- Gefcom's page claims that most competitive models reduced error i.e. RMSE by roughly <b>30%</b>